# FractalN Mushroom Classification

This notebook runs the training process in Google Colab using the main.py script.

## Setup
First, let's clone the repository and install dependencies:

In [ ]:
# Clone your repository (including dataset)
!git clone https://github.com/sapienskid/FractalN.git
%cd FractalN

# Install requirements
!pip install -r requirements.txt
!pip install colorama tqdm pillow numpy opencv-python cupy-cuda12x  # Ensure CUDA dependencies

# Verify dataset and source code
!ls data/processed_mushroom_dataset
!ls src

## GPU Check
Let's verify GPU availability:

In [ ]:
!nvidia-smi

In [ ]:
# Verify CuPy can access GPU
import cupy as cp

# Test GPU computation
x = cp.array([1, 2, 3])
print(f"CuPy GPU available: {cp.cuda.is_available()}")
if cp.cuda.is_available():
    device = cp.cuda.Device(0)
    print(f"GPU Device: {device.attributes['DEVICE_NAME'].decode('utf-8')}")
    print(f"Total Memory: {device.mem_info[1]/1024**3:.2f} GB")

# Test basic computation
y = cp.square(x)
print(f"\nTest computation on GPU: {x} squared = {y}")
print("GPU test successful!")

## Training
Now we'll run the training using main.py:

In [ ]:
import sys
sys.path.append('src')

from gpu_utils import configure_gpu
from main import train

# Configure GPU with memory pool
configure_gpu()

# Run your custom CNN implementation
train()